# **Pre-process Coronavirus Music Questionnaire (CMQ) data**
Lindsay Warrenburg & Lauren Fink  
lindsay.a.warrenburg@gmail.com  lauren.fink@ae.mpg.de  
Final version: December 20, 2020

The goal of this Jupyter Notebook is to clean the data from all different survey versions (languages) in Qualtrics.

We have around 1000 survey responses from Germany, Italy, France, India, New York, and the UK. Though the survey questions were almost identical in all versions, there are a few differences that need to be handled. 

We also need to get all question responses into useable formats and to remove any participants who were not taking the study seriously. 

# **Load required packages and modules**

In [1]:
# basic coding/ML tools
import numpy as np
import math
import pandas as pd
import os, sys, scipy, xlrd, urllib, itertools, re, warnings
from pandas.api.types import CategoricalDtype

# visualization tools
import matplotlib.pyplot as plt
import seaborn as sns, IPython.display
from IPython.display import Image, HTML
%matplotlib inline
plt.rcParams['figure.figsize'] = (14,4)

warnings.filterwarnings(action='ignore')

# Local imports
import local_dicts # dictionary defining column labels for our dataframe and answer text~answer code relationships 
import local_funcs # functions that we might want to use in multiple scripts throughout the project
import importlib

importlib.reload(local_dicts) 
importlib.reload(local_funcs)

# display all columns in dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load and combine data

In this section, we load all the Qualtrics dataframes:
1. Prolific data
2. Social Media data
3. Germany data
4. France data
5. Italy data
6. India data
7. US (NY) data 

We also fix known discrepencies between the different language versions. 

In [2]:
# Read in .csv files for each country
prolific = pd.read_csv('Raw_Data/prolific.csv')
social = pd.read_csv('Raw_Data/social.csv')
ger = pd.read_csv('Raw_Data/german.csv')
fra = pd.read_csv('Raw_Data/french.csv')
ita = pd.read_csv('Raw_Data/italian.csv')
ind = pd.read_csv('Raw_Data/indian.csv')
ny = pd.read_csv('Raw_Data/ny.csv')

# Add column to each datafram about source (for when we concatenate them later)
prolific['Survey Info_Source'] = 'Prolific'
social['Survey Info_Source'] = 'Social Media'
ger['Survey Info_Source'] = 'Ipsos_Germany'
fra['Survey Info_Source'] = 'Ipsos_France'
ita['Survey Info_Source'] = 'Ipsos_Italy'
ind['Survey Info_Source'] = 'Ipsos_India'
ny['Survey Info_Source'] = 'Ipsos_NY'

# For all of the Ipsos frames, remove data from people that were screened out 
# or kicked out because a quota was full
def cleanIpsos(df):
    nd = df.loc[df['Q_TerminateFlag'].astype(str) != 'Screened']
    nd = nd.loc[nd['Q_TerminateFlag'].astype(str) != 'QuotaMet']
    return nd

ger = cleanIpsos(ger)
fra = cleanIpsos(fra)
ita = cleanIpsos(ita)
ind = cleanIpsos(ind)
ny = cleanIpsos(ny)

# Fix column name issues specific to certain contries 
ny.rename(columns={'8c': '8c_state', '8c.1': '8c'}, inplace=True, errors="raise") # this aligns 8c to be ESM MIDDLE, like in other surveys

#ger.rename(columns={'33_3': '33_7', '33_4': '33_6', '33_5': '33_4', '33_6':'33_5', '33_7':'33_3'}, inplace=True, errors="raise")
#fra.rename(columns={'23_2': '23_3', '23_3': '23_2'}, inplace=True, errors="raise")
# TODO CHECK - does it do all at once because if not, we are re-writing duplicate columns
# do it serially to be safe
ger.rename(columns={'33_3': '33_7new', '33_4': '33_6new', '33_5': '33_4new', '33_6':'33_5new', '33_7':'33_3new'}, inplace=True, errors="raise")
ger.rename(columns={'33_7new':'33_7', '33_6new':'33_6', '33_4new':'33_4', '33_5new':'33_5', '33_3new':'33_3'}, inplace=True, errors="raise")

fra.rename(columns={'23_2': '23_3_new', '23_3': '23_2_new'}, inplace=True, errors="raise")
fra.rename(columns={'23_3_new':'23_3', '23_2_new':'23_2'}, inplace=True, errors="raise")

# Translate answer code values into years in school for prolific and social surveys. 
# (for other surveys this was numeric free response)
# In all other surveys, these were free numeric input
prolific['3'].iloc[2:] = prolific['3'].iloc[2:].astype(float).map(local_dicts.prolific_school_dict, na_action='ignore')
prolific['4'].iloc[2:] = prolific['4'].iloc[2:].astype(float).map(local_dicts.prolific_post_school_dict, na_action='ignore')
social['3'].iloc[2:] = social['3'].iloc[2:].astype(float).map(local_dicts.prolific_school_dict, na_action='ignore')
social['4'].iloc[2:] = social['4'].iloc[2:].astype(float).map(local_dicts.prolific_post_school_dict, na_action='ignore') 

# Fix issue of French version having different answer coding for Q 33
fix = [col for col in fra.columns if '33_' in col]
for i in fix:
    fra[i].iloc[2:] = fra[i].iloc[2:].map(local_dicts.french_Q33_dict, na_action='ignore')
    
# Fix issue of France and Italy having different gender orders
# this is question 2
fra['2'].iloc[2:] = fra['2'].iloc[2:].map(local_dicts.frenchItal_gender_dict, na_action='ignore')
ita['2'].iloc[2:] = ita['2'].iloc[2:].map(local_dicts.frenchItal_gender_dict, na_action='ignore')

# Fix issue of France and German having different answer codes for listening to music differently question
# this is question 16
fra['16'].iloc[2:] = fra['16'].iloc[2:].map(local_dicts.frenchGer_YN_musDiff_dict, na_action='ignore')
ger['16'].iloc[2:] = ger['16'].iloc[2:].map(local_dicts.frenchGer_YN_musDiff_dict, na_action='ignore')
    
# Print size of each data set
print("Size of each data frame:\n")
frames = [prolific, social, ger, fra, ita, ind, ny]
for df in frames: 
    print(df['Survey Info_Source'].iloc[0],':\t', df.shape)
    

Size of each data frame:

Prolific :	 (925, 209)
Social Media :	 (406, 209)
Ipsos_Germany :	 (1048, 206)
Ipsos_France :	 (1094, 218)
Ipsos_Italy :	 (1059, 219)
Ipsos_India :	 (1191, 215)
Ipsos_NY :	 (1131, 217)


The first value tells us the number of rows (i.e participants). The second value tells us the number of columns. Some of the extra columns in the different datasets are survey meta data that we do not need. Let's take a look at the column names. 

**Check that column names are the same between countries**

In [3]:
### Uncomment below if want to see column names 
# print(list(fra.columns))
# print('\n\n')
# print(list(prolific.columns))

# print(fra['ResponseId'].head(5))
### NOTE: Will used response ID in his cleaning

From the above, it is clear that we can use the column names as keys to concatenate all of the datasets. We will then deal with deleting irrelevant columns. See key-value mappings in 'local_dicts.py'

### Concatenate data frames from each source

In [4]:
# Use outerjoin to combine all tables.
# Outerjoin allows us to keep all columns and add NAs for missing columns in the other datasets
data = pd.concat(frames, axis=0, join='outer', sort=False)

### Uncomment below, if want to have a look at newly combined data frame
# print(data.shape)
# print('\n\n')
# print(data.head(5))
# print('\n\n')
# print(list(data.columns))

### Rename columns to have meaningful labels
Here, we rename the columns according to the survey questions, as defined in our imported dictionary. 

In [5]:
# drop columns we no longer need
data = data.drop(columns=local_dicts.dropCols)

# rename columns according to our column dictionary
data.rename(columns=local_dicts.column_dict, inplace=True)

# print(list(data.columns))
print(data.shape)

(6854, 206)


### Clean-up column types and labels
- Remove the first two rows of data, which correspond to the instructions.
- Write the instructions to a file for future use. 
- Remove any participants who got sent to the end of the survey because they were < 18
- Rearrange the columns to be in alphabetical order -- this way all the ***Music Listening*** columns are together and all the ***Making Music*** columns are together, etc.
- Convert numeric columns to numeric.
- Recode specific columns
- Remove any participants under 18 that got sent to end of survey
- Examine data type of our dataframe columns. 

In [6]:
# Remove first two rows (instructions)   
instructions = data.iloc[0,:] # keep these in case we want to look at question text later
instructions.to_csv(r'instructions.csv', index = None, header = True) # write to .csv
data = data.drop([0, 1])

# Convert all numeric columns to numeric. Exclude open and one hot columns
num_cols = []
for key, val in local_dicts.answer_code_dict.items(): 
    if val != 'open' and key not in local_dicts.oneHotCols:
        num_cols.append(key)
          
# Convert numeric      
data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce')

# Remove people < 18 from non-Ipsos data
# (there was no screen out flag for that), even though they got kicked to the last page
data = data.loc[data['Demographics_General_Age'] >= 18]

# Recode Ollen to be on proper scale (start at 1 instead of 0)
data['Demographics_Music_Ollen'] = data['Demographics_Music_Ollen'] + 1

# Recode number of times leaving house
# This question starts with the answer 0 with code 1, so we need to subtract 1 from the column to get in actual values
data["Demographics_COVID_Number Times Leave"] = data['Demographics_COVID_Number Times Leave'] - 1

# Turn survey duration into minutes (from seconds)
data["Survey Info_Duration"] = data['Survey Info_Duration'].astype(float)/60

# Re-index
data.reset_index(inplace = True, drop = True) 

# Rearrange columns to be in alphabetical order
data = data.reindex(sorted(data.columns), axis=1)

### column types
# local_funcs.View(data.dtypes.to_frame())

### Concatenate unique codes for ESM study
We need to keep track of each person's self-generated ESM code.

In [7]:
esm = [col for col in data.columns if col.startswith('ESM_')]
esm.remove('ESM_ID') # don't touch the Qualtrics ID

# Clean text in ESM cols
for col in esm:
    data[col] = data[col].str.strip()
    data[col] = data[col].str.upper()
    
# Combine code into one ESM code
data['ESM_Self-Generated Code'] = data['ESM_Mother'] + data['ESM_Father'] + data['ESM_Middle'] + data['ESM_Town']

# Drop the other columns we no longer need
data = data.drop(['ESM_Mother', 'ESM_Father', 'ESM_Middle', 'ESM_Town'], axis=1)

### Clean up text data

#### Free Responses
First, look at the raw responses.

In [8]:
free = [col for col in data.columns if col.endswith('Free Response')]
# for column in free:
#     print("Column:", column)
#     print(data[column].unique())
#     print("\n \n---------------------")

In [9]:
for column in free: 
    local_funcs.textCleaning(data, column)
#     print("Column:", column, "\n")
#     print(data[column].unique())
#     print("\n \n---------------------")

There's one write-in response (`Profession`) that we can clean a bit, too.

In [10]:
# Demographics_COVID_Profession
data['Demographics_COVID_Profession'] = data['Demographics_COVID_Profession'].str.strip()
data['Demographics_COVID_Profession'] = data['Demographics_COVID_Profession'].str.lower()

### Create One-Hot Encoded Columns
When you have a survey question that allows you to pick several options (like a `Check all that apply` question), you get some answers with only one "checked" item and other answers with many "checked" items. It's impossible to analyze responses this way.

**Instead, we need to create separate columns for each possible item. For example"**
- For the `Living Situation` questions, a person may respond `I live with a pet` and `I live with a child.` 
- We should create a *Yes/No* column for `I live with a pet` and `I live with a child.`
- Some people will have many items checked (multiple "yes" responses), whereas others will have no items or 1 item selected.
- I also created a `No Response` column to distinguish those who left the question blank.

In [11]:
# Demographics_COVID_Living Situation 
# recode cousin to 0
data['Demographics_COVID_Living Situation'] = data['Demographics_COVID_Living Situation'].str.replace('10','0')

data['Demographics_COVID_Living Situation-Alone'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('1', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Pet'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('2', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Partner'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('3', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Child'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('4', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Parent'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('5', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Elderly'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('6', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Friend'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('7', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Shared'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('8', na=False), 1, 0)
data['Demographics_COVID_Living Situation-Other'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('9', na=False), 1, 0)
data['Demographics_COVID_Living Situation-SibCous'] = np.where(data['Demographics_COVID_Living Situation'].str.contains('0', na=False), 1, 0)
data['Demographics_COVID_Living Situation-No Response'] = pd.isnull(data['Demographics_COVID_Living Situation']) * 1

Let's compare the original column *Demographics_COVID_Living Situation* with the new *Yes/No* columns.

In [12]:
# data.loc[:, ['Demographics_COVID_Living Situation',
#              'Demographics_COVID_Living Situation-Alone',
#              'Demographics_COVID_Living Situation-Pet',
#              'Demographics_COVID_Living Situation-Partner',
#              'Demographics_COVID_Living Situation-Child',
#              'Demographics_COVID_Living Situation-Parent',
#              'Demographics_COVID_Living Situation-Elderly',
#              'Demographics_COVID_Living Situation-Friend',
#              'Demographics_COVID_Living Situation-Shared',
#              'Demographics_COVID_Living Situation-Other',
#              'Demographics_COVID_Living Situation-SibCous',
#              'Demographics_COVID_Living Situation-No Response']]

We now repeat this process for other questions!

In [13]:
# Demographics_COVID_Ways Work Affected
data['Demographics_COVID_Ways Work Affected-Still Go To Work'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('1', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Still Work From Home'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('2', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Now Work From Home'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('3', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Reduced Hours'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('4', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Lower Income'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('5', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Lost Job'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('6', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Job At Risk'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('7', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-Homeschool Children'] = np.where(data['Demographics_COVID_Ways Work Affected'].str.contains('8', na=False), 1, 0)
data['Demographics_COVID_Ways Work Affected-No Response'] = pd.isnull(data['Demographics_COVID_Ways Work Affected']) * 1

# data.loc[:, ['Demographics_COVID_Ways Work Affected',
#              'Demographics_COVID_Ways Work Affected-Still Go To Work',
#              'Demographics_COVID_Ways Work Affected-Still Work From Home',
#              'Demographics_COVID_Ways Work Affected-Now Work From Home',
#              'Demographics_COVID_Ways Work Affected-Reduced Hours',
#              'Demographics_COVID_Ways Work Affected-Lower Income',
#              'Demographics_COVID_Ways Work Affected-Lost Job',
#              'Demographics_COVID_Ways Work Affected-Job At Risk',
#              'Demographics_COVID_Ways Work Affected-Homeschool Children',
#              'Demographics_COVID_Ways Work Affected-No Response']]

In [14]:
# Demographics_Health_Infected with COVID
data['Demographics_Health_Infected with COVID-I Am Infected'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('1', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-I Live With Someone'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('2', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-I Have Professional Contact'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('3', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-Someone I Am Close To'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('4', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-I Am Mourning A Loss'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('5', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-None'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('0', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-Prefer Not To Say'] = np.where(data['Demographics_Health_Infected with COVID'].str.contains('99', na=False), 1, 0)
data['Demographics_Health_Infected with COVID-No Response'] = pd.isnull(data['Demographics_Health_Infected with COVID']) * 1

# data.loc[:, ['Demographics_Health_Infected with COVID',
#              'Demographics_Health_Infected with COVID-I Am Infected',
#              'Demographics_Health_Infected with COVID-I Live With Someone',
#              'Demographics_Health_Infected with COVID-I Have Professional Contact',
#              'Demographics_Health_Infected with COVID-Someone I Am Close To',
#              'Demographics_Health_Infected with COVID-I Am Mourning A Loss',
#              'Demographics_Health_Infected with COVID-Prefer Not To Say',
#              'Demographics_Health_Infected with COVID-No Response']]

**Since we have the one-hot encoded columns now, we can delete the original columns with text responses.**

In [15]:
# drop fixed multi-choice columns
data = data.drop(['Demographics_COVID_Living Situation', 
                  'Demographics_COVID_Ways Work Affected',
                  'Demographics_Health_Infected with COVID'], 
                 axis=1)

print("data.shape: ", data.shape)

data.shape:  (6677, 228)


### More one hot encoding

In [16]:
# OHE employment before covid
data['Demographics_General_Employement Before COVID-Employed Full Time'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 4, 1, 0)
data['Demographics_General_Employement Before COVID-Employed Part Time'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 5, 1, 0)
data['Demographics_General_Employement Before COVID-Self Employed'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 6, 1, 0)
data['Demographics_General_Employement Before COVID-Student'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 7, 1, 0)
data['Demographics_General_Employement Before COVID-Home Maker Caregiver'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 8, 1, 0)
data['Demographics_General_Employement Before COVID-Retired'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 10, 1, 0)
data['Demographics_General_Employement Before COVID-Prefer Not To Say'] = np.where(data['Demographics_COVID_Employement Before COVID'] == 11, 1, 0)
data['Demographics_General_Employement Before COVID-No Response'] = pd.isnull(data['Demographics_COVID_Employement Before COVID']) * 1

data = data.drop(['Demographics_COVID_Employement Before COVID'], axis=1)

In [17]:
# OHE Generally make music alone or together
data['Demographics_Music_Making Music-Alone'] = np.where(data['Demographics_Music_Make Music Alone or Together'] == 1, 1, 0)
data['Demographics_Music_Making Music-Together'] = np.where(data['Demographics_Music_Make Music Alone or Together'] == 2, 1, 0)
data['Demographics_Music_Making Music-Both Alone And Together'] = np.where(data['Demographics_Music_Make Music Alone or Together'] == 3, 1, 0)
data['Demographics_Music_Making Music-No Response'] = pd.isnull(data['Demographics_Music_Make Music Alone or Together']) * 1

data = data.drop(['Demographics_Music_Make Music Alone or Together'], axis=1)

In [18]:
# OHE current city type 
data['Demographics_General_City Type-Rural'] = np.where(data['Demographics_COVID_Current City Type'] == 4, 1, 0)
data['Demographics_General_City Type-Suburban'] = np.where(data['Demographics_COVID_Current City Type'] == 5, 1, 0)
data['Demographics_General_City Type-Urban'] = np.where(data['Demographics_COVID_Current City Type'] == 6, 1, 0)
data['Demographics_General_City Type-No Response'] = pd.isnull(data['Demographics_COVID_Current City Type']) * 1

data = data.drop(['Demographics_COVID_Current City Type'], axis=1)

In [19]:
# OHE type of music listened to during covid
data['Music Engagement_Music Types-Same Musicians Composers'] = np.where(data['Music Engagement_Engaging With Music Differently List'] == 11, 1, 0)
data['Music Engagement_Music Types-Other Musicians Composers'] = np.where(data['Music Engagement_Engaging With Music Differently List'] == 12, 1, 0)
data['Music Engagement_Music Types-Other Genres'] = np.where(data['Music Engagement_Engaging With Music Differently List'] == 13, 1, 0)
data['Music Engagement_Music Types-No Response'] = pd.isnull(data['Music Engagement_Engaging With Music Differently List']) * 1

data = data.drop(['Music Engagement_Engaging With Music Differently List'], axis=1)

### Adding summary columns

In [20]:
# Create column for total years of education 
data['Demographics_General_Education Years'] = data['Demographics_General_Years of School'].fillna(0) + data['Demographics_General_Years of Post-School'].fillna(0)


# Create column for covid contact of any kind
data['Demographics_Health_COVID Contact Binary'] = 0
data['Demographics_Health_COVID Contact Binary'] = np.where( (data['Demographics_Health_Infected with COVID-I Am Infected']==1)
                                                            | (data['Demographics_Health_Infected with COVID-I Live With Someone']==1) 
                                                            | (data['Demographics_Health_Infected with COVID-I Have Professional Contact']==1)
                                                            | (data['Demographics_Health_Infected with COVID-Someone I Am Close To']==1)
                                                            | (data['Demographics_Health_Infected with COVID-I Am Mourning A Loss']==1), 
                                                            1, data['Demographics_Health_COVID Contact Binary'])

# Create column for general change in work situation
data['Demographics_COVID_Work Changed Binary'] = 0
data['Demographics_COVID_Work Changed Binary'] = np.where( (data['Demographics_COVID_Ways Work Affected-Now Work From Home']==1) 
                                                          | (data['Demographics_COVID_Ways Work Affected-Reduced Hours']==1) 
                                                          | (data['Demographics_COVID_Ways Work Affected-Job At Risk']==1)
                                                          | (data['Demographics_COVID_Ways Work Affected-Lower Income']==1)
                                                          | (data['Demographics_COVID_Ways Work Affected-Lost Job']==1), 
                                                          1, data['Demographics_COVID_Work Changed Binary'])

### Score Big 5 Personality Dimensions
Here, we recode ten questions from the **10-item Big 5 Personality Questionnaire** into their five components:
- Extraversion
- Aggreeableness
- Conscientiousness
- Neuroticism
- Openness to experience

In [21]:
# Define reverse dictionary for items that need to be reverse scored. 
reverse_dict = {"1.0":7, "2.0":6, "3.0":5, "4.0":4, "5.0":3, "6.0":2, "7.0":1}

# Extraversion = Demographics_Personality_Outgoing + Demographics_Personality_Reserved -- REVERSE SCORED
data['Demographics_Personality_Reserved_rs'] = data['Demographics_Personality_Reserved'].map(str).map(reverse_dict)
data['Demographics_Personality_Extraversion'] = data[['Demographics_Personality_Outgoing', 'Demographics_Personality_Reserved_rs']].mean(axis=1)

# # Agreeableness = Demographics_Personality_Trusting + Demographics_Personality_Faults Others -- REVERSE SCORED
data['Demographics_Personality_Faults Others_rs'] = data['Demographics_Personality_Faults Others'].map(str).map(reverse_dict)
data['Demographics_Personality_Agreeableness'] = data[['Demographics_Personality_Trusting', 'Demographics_Personality_Faults Others_rs']].mean(axis=1)

# # Conscientiousness = Demographics_Personality_Thorough + Demographics_Personality_Lazy -- REVERSE SCORED
data['Demographics_Personality_Lazy_rs'] = data['Demographics_Personality_Lazy'].map(str).map(reverse_dict)
data['Demographics_Personality_Conscientiousness'] = data[['Demographics_Personality_Thorough', 'Demographics_Personality_Lazy_rs']].mean(axis=1)

# # Neuroticism = Demographics_Personality_Nervous + Demographics_Personality_Relaxed -- REVERSE SCORED
data['Demographics_Personality_Relaxed_rs'] = data['Demographics_Personality_Relaxed'].map(str).map(reverse_dict)
data['Demographics_Personality_Neuroticism'] = data[['Demographics_Personality_Nervous', 'Demographics_Personality_Relaxed_rs']].mean(axis=1)

# # Openness to experience = Demographics_Personality_Imaginative + Demographics_Personality_Few Artistic Interests -- REVERSE SCORED
data['Demographics_Personality_Few Artistic Interests_rs'] = data['Demographics_Personality_Few Artistic Interests'].map(str).map(reverse_dict)
data['Demographics_Personality_Openness'] = data[['Demographics_Personality_Imaginative', 'Demographics_Personality_Few Artistic Interests_rs']].mean(axis=1)

Let's see how it worked. I made a table where the new dimension (e.g., Extraversion) is followed by the two corresponding questions from the questionnaire.

In [22]:
# data.loc[:, ['Demographics_Personality_Extraversion',
#              'Demographics_Personality_Outgoing',
#              'Demographics_Personality_Reserved_rs',
             
#              'Demographics_Personality_Agreeableness',
#              'Demographics_Personality_Trusting',
#              'Demographics_Personality_Faults Others_rs',
             
#              'Demographics_Personality_Conscientiousness',
#              'Demographics_Personality_Thorough',
#              'Demographics_Personality_Lazy_rs',
            
#              'Demographics_Personality_Neuroticism',
#              'Demographics_Personality_Nervous',
#              'Demographics_Personality_Relaxed_rs',
            
#              'Demographics_Personality_Openness',
#              'Demographics_Personality_Imaginative',
#              'Demographics_Personality_Few Artistic Interests_rs']]

Since we have created columns for the five personality dimensions, we can remove each of the individual questions from our dataframe.

In [23]:
# drop personality
data = data.drop(['Demographics_Personality_Outgoing', 'Demographics_Personality_Reserved', 
                  'Demographics_Personality_Trusting', 'Demographics_Personality_Faults Others',
                  'Demographics_Personality_Thorough', 'Demographics_Personality_Lazy',
                  'Demographics_Personality_Nervous', 'Demographics_Personality_Relaxed',
                  'Demographics_Personality_Imaginative', 'Demographics_Personality_Few Artistic Interests',
                  'Demographics_Personality_Few Artistic Interests_rs', 'Demographics_Personality_Relaxed_rs',  
                  'Demographics_Personality_Lazy_rs', 'Demographics_Personality_Faults Others_rs', 
                  'Demographics_Personality_Reserved_rs'
                 ], 
                 axis=1)

print("data.shape: ", data.shape)

data.shape:  (6677, 242)


# Country-specific cleaning

### Remove data not from 6 countries of interest
Remove data from countries for which we don't have enough samples
Only proceed with subsamples for which we have enough data and which were sampled in an unbiased way. (i.e remove social media sample and remove people in the prolific sample coming from countries outside of UK). We decided as a group that we wanted to save the social media dataset for a different paper and only use data from representative samples for this paper. 

In [24]:
# Only keep data from big 6 countries and representative samples 
# Filter by source, country code, and state code (if relevant)
# output individual tables for each country, as well as one concatenated table
ger = data.loc[data['Survey Info_Source'] == 'Ipsos_Germany']
ger = ger.loc[ger['Demographics_COVID_Current Country'] == 3418]

fra = data.loc[data['Survey Info_Source'] == 'Ipsos_France']
fra = fra.loc[fra['Demographics_COVID_Current Country'] == 3414]

ita = data.loc[data['Survey Info_Source'] == 'Ipsos_Italy']
ita = ita.loc[ita['Demographics_COVID_Current Country'] == 3436]

ind = data.loc[data['Survey Info_Source'] == 'Ipsos_India']
ind = ind.loc[ind['Demographics_COVID_Current Country'] == 3430]

ny = data.loc[data['Survey Info_Source'] == 'Ipsos_NY']
ny = ny.loc[ny['Demographics_COVID_Current Country'] == 3537]
ny = ny.loc[ny['Demographics_COVID_Current State'] == '33']

uk = data.loc[data['Survey Info_Source'] == 'Prolific']
uk = uk.loc[uk['Demographics_COVID_Current Country'] == 3535]

frames = [ger, fra, ita, ind, ny, uk]

    
# concatenate all into one df again
b6 = pd.concat(frames, axis=0, join='outer', sort=False)

In [25]:
print(b6['Demographics_COVID_Current Country'].unique())
pd.DataFrame(b6['Survey Info_Source'].value_counts())

[3418. 3414. 3436. 3430. 3537. 3535.]


,Survey Info_Source
Ipsos_India,1098
Ipsos_France,1059
Ipsos_NY,1043
Ipsos_Italy,1029
Ipsos_Germany,1024
Prolific,654


In [26]:
# overwrite our data variable before proceeding
data = b6.copy()
data.shape

(5907, 242)

### Add government indices for each of our 6 countries
The Oxford group was tracking government responses to the pandemic situation and has made their data freely available. We are interested in related our survey responses to the severity of lockdown measures in place at the time in each country.

More info about the OxCGRT:
https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/index_methodology.md

https://www.bsg.ox.ac.uk/research/research-projects/coronavirus-government-response-tracker


In [27]:
og = pd.read_csv('OxCGRT_latest_23062020.csv') 

In [28]:
# Explore and clean OG a bit
# print(og.columns)

# Subset OG to only have data from our countries of interest
og = og[og['CountryName'].isin(['Germany', 'Italy', 'France', 'India', 'United Kingdom', 'United States']) ]
print(og['CountryName'].unique())

# Drop columns we don't need
for c in og.columns:
    if 'Flag' in c: 
        og = og.drop(columns = c)
#     if 'ForDisplay' in c: # display might actually be what we want because it smoothes over 7 days
#         og = og.drop(columns = c)
og = og.drop(columns = ['CountryCode', 'M1_Wildcard', 'StringencyLegacyIndex'])        
        
print(og.columns)

['Germany' 'France' 'United Kingdom' 'India' 'Italy' 'United States']
Index(['CountryName', 'Date', 'C1_School closing', 'C2_Workplace closing',
       'C3_Cancel public events', 'C4_Restrictions on gatherings',
       'C5_Close public transport', 'C6_Stay at home requirements',
       'C7_Restrictions on internal movement',
       'C8_International travel controls', 'E1_Income support',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'GovernmentResponseIndexForDisplay',
       'ContainmentHealthIndex', 'ContainmentHealthIndexForDisplay',
       'EconomicSupportIndex', 'EconomicSupportIndexForDisplay'],
      dtype='obj

In [29]:
# Make date reporting formats compatible between the two tables because we need to join based on date. 
# We will use the end date for our survey responses
colName = data.columns.get_loc('Survey Info_EndDate')
data.iloc[:,colName] = data.iloc[:,colName].str.split(None, 1)
data.iloc[:,colName] = data.iloc[:,colName].map(lambda x: x[0])
data.iloc[:,colName] = data.iloc[:,colName].str.replace("-", "")
data['Survey Info_EndDate'] = data['Survey Info_EndDate'].astype(int)

# print(data['Survey Info_EndDate'].head(5))
# print(og['Date'].head(5))

In [30]:
# Make country labels compatible between the two data frames
data['Demographics_COVID_Current Country'] = data['Demographics_COVID_Current Country'].map(local_dicts.answer_code_dict['Demographics_COVID_Current Country'])
print(data['Demographics_COVID_Current Country'].unique())
print(og['CountryName'].unique())

og['CountryName'] = og['CountryName'].map({'United Kingdom': 'UK', 'United States': 'USA', 'France': 'France', 'Italy': 'Italy', 'India':'India', 'Germany':'Germany'})
print(data['Demographics_COVID_Current Country'].unique())
print(og['CountryName'].unique())

['Germany' 'France' 'Italy' 'India' 'USA' 'UK']
['Germany' 'France' 'United Kingdom' 'India' 'Italy' 'United States']
['Germany' 'France' 'Italy' 'India' 'USA' 'UK']
['Germany' 'France' 'UK' 'India' 'Italy' 'USA']


In [31]:
### Add Oxford data to our table, based on date and country

### Get col names from Oxford data
# new_cols = og.columns.tolist()

### Append OG so we can keep them straight
# new_cols = ['OG_' + s for s in new_cols]

### Add empty cols to our data frame
# data = data.reindex(columns=data.columns.tolist() + new_cols) 

### rearrange columns to be in alphabetical order
# data = data.reindex(sorted(data.columns), axis=1)

### Merge data frames
data = pd.merge(data, og,  how='left', left_on=['Demographics_COVID_Current Country','Survey Info_EndDate'], right_on = ['CountryName','Date'])

In [32]:
data['date'] = pd.to_datetime(data['Survey Info_EndDate'], format='%Y%m%d', errors='ignore')

In [33]:
data.rename(columns={'ConfirmedCases':'Country_Confirmed Cases',
                     'ConfirmedDeaths':'Country_Confirmed Deaths',
                     'ContainmentHealthIndexForDisplay': 'Country_Containment Health Index',
                     'EconomicSupportIndexForDisplay':'Country_Economic Support Index',
                     'GovernmentResponseIndexForDisplay':'Country_Government Response Index',
                     'StringencyIndexForDisplay':'Country_Stringency Index',
                     'Demographics_COVID_Current Country':'Country_Country Name'
                    }, 
            inplace=True)

In [34]:
data = data.reindex(sorted(data.columns), axis=1)
data['ID'] = data.index + 1
id_col = data.pop('ID')
data.insert(0, 'ID', id_col)
data.reset_index(inplace = True, drop = True)

In [35]:
# Drop columns that we're not using
drop_cols = [
         'C1_School closing', 'C2_Workplace closing', 'C3_Cancel public events',
         'C4_Restrictions on gatherings', 'C5_Close public transport', 'C6_Stay at home requirements',
         'C7_Restrictions on internal movement', 'C8_International travel controls',
         'ContainmentHealthIndex', 
         'CountryName', 'Date',
         'Demographics_COVID_Current State', # only NYC
         'Demographics_COVID_Profession', # free text
         'Demographics_General_Years of Post-School', 'Demographics_General_Years of School', # combined in the 'Demographics_General_Education Years column'
         'E1_Income support', 'E2_Debt/contract relief', 'E3_Fiscal measures', 'E4_International support',
         'ESM_Self-Generated Code', 
         'EconomicSupportIndex', 
         'GovernmentResponseIndex', 
         'H1_Public information campaigns', 'H2_Testing policy', 
         'H3_Contact tracing', 'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
         'ResponseId', 
         'StartDate', 
         'StringencyIndex', 'StringencyLegacyIndexForDisplay',
         'Survey Info_EndDate', 
         'Survey Info_Progress', 'Survey Info_RecordedDate', 'Survey Info_Source'
        ]

data = data.drop(columns = drop_cols, axis = 1)
data.reset_index(inplace = True, drop = True) 

# Export quantitative data

In [36]:
data.shape

(5907, 239)

In [37]:
data.to_csv(r'raw_data.csv', index = None, header = True)